<a href="https://colab.research.google.com/github/osmarbraz/exemplos_BERT/blob/main/ExemplosPrevisaoProximaSentencaBERT_pt_br.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Previsão Próxima Sentença usando BERT Transformers by HuggingFace

## **A execução pode ser feita através do menu Ambiente de Execução opção Executar tudo.**

Exemplos **Previsão Próxima Sentença** usando **BERT**.

**Link biblioteca Huggingface:**
https://github.com/huggingface/transformers


**Artigo original BERT Jacob Devlin:**
https://arxiv.org/pdf/1506.06724.pdf

## 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

###Tratamento de logs

Método para tratamento dos logs.

In [ ]:
#biblioteca de logging
import logging

#formatando a mensagem de logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
#se estiver executando no Google Colaboratory
import sys
#retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1 - Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [ ]:
# Instala a última versão da biblioteca
#!pip install transformers

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.5.1

     |████████████████████████████████| 2.1 MB 14.2 MB/s 
     |████████████████████████████████| 3.3 MB 50.4 MB/s 
     |████████████████████████████████| 895 kB 50.2 MB/s 


## 2 - Download do arquivo do PyTorch Checkpoint

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **'neuralmind/bert-base-portuguese-cased'**
* **'neuralmind/bert-large-portuguese-cased'**

In [ ]:
# Importando as bibliotecas
import os

# Variável para setar o arquivo
URL_MODELO = None

# Comente uma das urls para carregar modelos de tamanhos diferentes(base/large)
# URL_MODELO do arquivo do modelo tensorflow
# arquivo menor(base) 1.1 Gbytes
#URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip"

# arquivo grande(large) 3.5 Gbytes
URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip"

# Se a variável foi setada
if URL_MODELO:

    # Diretório descompactação
    DIRETORIO_MODELO = '/content/modelo'

    # Recupera o nome do arquivo do modelo da URL_MODELO
    arquivo = URL_MODELO.split("/")[-1]

    # Nome do arquivo do vocabulário
    arquivo_vocab = "vocab.txt"

    # Caminho do arquivo na URL_MODELO
    caminho = URL_MODELO[0:len(URL_MODELO)-len(arquivo)]

    # Verifica se a pasta de descompactação existe na pasta corrente
    if os.path.exists(DIRETORIO_MODELO):
      print("Apagando diretório existente do modelo!")
      # Apaga a pasta e os arquivos existentes
      !rm -rf $DIRETORIO_MODELO    

    # Baixa o arquivo do modelo
    !wget $URL_MODELO
    
    # Descompacta o arquivo na pasta de descompactação
    !unzip -o $arquivo -d $DIRETORIO_MODELO

    # Baixa o arquivo do vocabulário
    # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente
    URL_MODELO_VOCAB = caminho + arquivo_vocab
    !wget $URL_MODELO_VOCAB
    
    # Coloca o arquivo do vocabulário no diretório de descompactação
    !mv $arquivo_vocab $DIRETORIO_MODELO
            
    # Move o arquivo para pasta de descompactação
    !mv $arquivo $DIRETORIO_MODELO
       
    print('Pasta do ' + DIRETORIO_MODELO + ' pronta!')
    
    # Lista a pasta corrente
    !ls -la $DIRETORIO_MODELO
else:
    print('Variável URL_MODELO não setada!')

--2021-12-10 20:26:30--  https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip
Resolving neuralmind-ai.s3.us-east-2.amazonaws.com (neuralmind-ai.s3.us-east-2.amazonaws.com)... 52.219.104.176
Connecting to neuralmind-ai.s3.us-east-2.amazonaws.com (neuralmind-ai.s3.us-east-2.amazonaws.com)|52.219.104.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1244275810 (1.2G) [application/zip]
Saving to: ‘bert-large-portuguese-cased_pytorch_checkpoint.zip’

bert-large-portugue 100%[===================>]   1.16G  30.1MB/s    in 43s     

2021-12-10 20:27:13 (27.6 MB/s) - ‘bert-large-portuguese-cased_pytorch_checkpoint.zip’ saved [1244275810/1244275810]

Archive:  bert-large-portuguese-cased_pytorch_checkpoint.zip
  inflating: /content/modelo/config.json  
  inflating: /content/modelo/pytorch_model.bin  
--2021-12-10 20:27:29--  https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-po

## 3 - Carregando o Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

Carregando o tokenizador da pasta '/content/modelo/' do diretório padrão se variável `URL_MODELO` setada.

**Caso contrário carrega da comunidade**

Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.

O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado apartir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

In [ ]:
# Importando as bibliotecas do tokenizador
from transformers import BertTokenizer

tokenizer = None

# Se a variável URL_MODELO foi setada
if URL_MODELO:
    # Carregando o Tokenizador
    print('Carrgando o tokenizador BERT do diretório ' + DIRETORIO_MODELO + '...')

    tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, 
                                              do_lower_case=False)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o tokenizador da comunidade...')
    
    #tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
    tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False)

Carrgando o tokenizador BERT do diretório /content/modelo...


## 4 - Carregando o Modelo BERT(BertForNextSentencePrediction)

Se a variável `URL_MODELO` estiver setada carrega o modelo do diretório `content/modelo`.

Caso contrário carrega da comunidade.

Carregando o modelo da pasta '/content/modelo/' do diretório padrão.

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

A documentação para estas pode ser encontrada em [aqui](https://huggingface.co/transformers/model_doc/bert.html#bertfornextsentenceprediction).

Por default o modelo está em modo avaliação ou seja `model.eval()`.

-----------------------

Durante a avaliação do modelo, este retorna um número de diferentes objetos com base em como é configurado na chamada do método `from_pretrained`. 

Quando definimos `output_hidden_states = True` na chamada do método `from_pretrained`, retorno do modelo possui no terceiro item os estados ocultos(**hidden_states**) de todas as camadas.  Veja a documentação para mais detalhes: https://huggingface.co/transformers/model_doc/bert.html#bertmodel

Quando **`output_hidden_states = True`** model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output; 
- outputs[2] = hidden_states.

Quando **`output_hidden_states = False`** ou não especificado model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output.


**ATENÇÃO**: O parâmetro ´**output_hidden_states = True**´ habilita gerar as camadas ocultas do modelo. Caso contrário somente a última camada é mantida. Este parâmetro otimiza a memória mas não os resultados.


In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BertForNextSentencePrediction

# Se a variável URL_MODELO1 foi setada
if URL_MODELO:
    # Carregando o modelo
    print('Carregando o modelo BERT do diretório ' + DIRETORIO_MODELO + '...')

    model = BertForNextSentencePrediction.from_pretrained(DIRETORIO_MODELO, 
                                      output_hidden_states = True)    
else:
    # Carregando o modelo da comunidade
    print('Carregando o modelo BERT da comunidade ...')

    model = BertForNextSentencePrediction.from_pretrained('neuralmind/bert-large-portuguese-cased', 
                                       output_hidden_states = True)
    

Carregando o modelo BERT do diretório /content/modelo...


Some weights of the model checkpoint at /content/modelo were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.eval()

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

## 5 - Função de previsão da próxima sentença com probabilidades

In [ ]:
import torch

def pontuacaoSentenca(entrada, candidato):			
  # Adiciona os tokens especiais ao texto
  texto_marcado = "[CLS] " + entrada + " [SEP] " + candidato + " [SEP]"
  #print("texto_marcado:", texto_marcado)
  
  # Divide a sentenças em tokens
  texto_tokenizado = tokenizer.tokenize(texto_marcado)    
  #print("texto_tokenizado:", texto_tokenizado

  # Mapeia os tokens em seus índices do vocabulario
  tokens_indexados = tokenizer.convert_tokens_to_ids(texto_tokenizado)
  #print("tokens_indexados:", tokens_indexados)

  # Converte as entradas de lista para tensores do torch
  tokens_tensores = torch.tensor([tokens_indexados])
  
  with torch.no_grad():
	  # Submete o texto ao modela
    outputs = model(tokens_tensores)  
	# outputs é uma matriz com perdas como o primeiro valor e logits como o segundo
  # outputs[0] = last_hidden_state
  # outputs[0][0] é igual outputs.logits[0]. Logits retorna dois valores, referenciado pelo terceiro índice.      
  # A função `.item()` retorna apenas o valor Python do tensor.       
  print("Saída:",outputs[0][0])
  
  # Calcula a média das probabildades
  pontuacao = torch.mean(outputs.logits[0])   
  #print(candidato, ":", pontuacao)
  return pontuacao

## 6 - Exemplos

### Comparando sentenças

In [ ]:
sentenca1 = "Hoje está calor."
sentenca2 = "Vamos a praia."
sentenca3 = "O carro estragou."
sentenca4 = "Vou levar a oficina."

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
resultado = pontuacaoSentenca(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)

print("\nSentença 1:", sentenca1)
print("Sentença 3:", sentenca3)
resultado = pontuacaoSentenca(sentenca1, sentenca3)
print("Resultado da relação da sentença 3:", resultado)

print("\nSentença 3:", sentenca3)
print("Sentença 4:", sentenca4)
resultado = pontuacaoSentenca(sentenca3, sentenca4)
print("Resultado da relação da sentença 4:", resultado)

print("\nSentença 3:", sentenca3)
print("Sentença 1:", sentenca1)
resultado = pontuacaoSentenca(sentenca3, sentenca1)
print("Resultado da relação da sentença 1:", resultado)


Sentença 1: Hoje está calor.
Sentença 2: Vamos a praia.
Saída: tensor([ 0.6251, -0.9251])
Resultado da relação da sentença 2: tensor(-0.1500)

Sentença 1: Hoje está calor.
Sentença 3: O carro estragou.
Saída: tensor([0.1169, 0.5731])
Resultado da relação da sentença 3: tensor(0.3450)

Sentença 3: O carro estragou.
Sentença 4: Vou levar a oficina.
Saída: tensor([ 0.9069, -1.0739])
Resultado da relação da sentença 4: tensor(-0.0835)

Sentença 3: O carro estragou.
Sentença 1: Hoje está calor.
Saída: tensor([ 0.2409, -0.0578])
Resultado da relação da sentença 1: tensor(0.0916)


### Comparando uma sequência de sentenças 1

In [ ]:
# Define um texto
texto = ['Fui de carro.',
         'O caminhão tombou.',              
         'O ônibus atrasou.']

total = 0
for i in range(len(texto)-1):
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))

Saída: tensor([ 0.8284, -0.1576])
Fui de carro.  =>  O caminhão tombou. : tensor(0.3354)
Saída: tensor([ 0.5830, -0.0621])
O caminhão tombou.  =>  O ônibus atrasou. : tensor(0.2605)
Total sentenças: 3
Total: tensor(0.5959)
Média: tensor(0.1986)


### Comparando uma sequência de sentenças 2

In [ ]:
# Define um texto com a permutação das sentenças do texto original
texto = ['O gato saiu correndo.',   
             'O cachorro latiu.',             
             'O passáro está no ninho.']    

total = 0
for i in range(len(texto)-1):
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Saída: tensor([0.1706, 0.7546])
O gato saiu correndo.  =>  O cachorro latiu. : tensor(0.4626)
Saída: tensor([-0.0530, -0.1603])
O cachorro latiu.  =>  O passáro está no ninho. : tensor(-0.1066)
Total sentenças: 3
Total: tensor(0.5692)
Média: tensor(0.1897)


### Comparando texto original com permutado

In [ ]:
print("Texto Original")

# Define um texto com 4 sentenças
texto_original = ['Bom Dia, professor.',
             'Qual o conteúdo da prova?',              
             'Vai cair tudo na prova?',
             'Aguardo uma resposta, João.']

texto = texto_original

total = 0
for i in range(len(texto)-1):    
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + pontuacao
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

print("\nTexto permutado")
# Define um texto com a permutação das sentenças do texto original
texto = [texto_original[3],   # 'Aguardo uma resposta, João.',
             texto_original[1],             # 'Qual o conteúdo da prova?',              
             texto_original[0],             # 'Vai cair tudo na prova?',
             texto_original[2]]             # 'Bom Dia, professor.']                 

total = 0
for i in range(len(texto)-1):
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Texto Original
Saída: tensor([ 1.5311, -0.2564])
Bom Dia, professor.  =>  Qual o conteúdo da prova? : tensor(0.6373)
Saída: tensor([0.8600, 0.5956])
Qual o conteúdo da prova?  =>  Vai cair tudo na prova? : tensor(0.7278)
Saída: tensor([ 1.4897, -1.0947])
Vai cair tudo na prova?  =>  Aguardo uma resposta, João. : tensor(0.1975)
Total sentenças: 4
Total: tensor(1.5627)
Média: tensor(0.3907)

Texto permutado
Saída: tensor([ 1.5511, -0.4897])
Aguardo uma resposta, João.  =>  Qual o conteúdo da prova? : tensor(0.5307)
Saída: tensor([ 1.0969, -0.0118])
Qual o conteúdo da prova?  =>  Bom Dia, professor. : tensor(0.5425)
Saída: tensor([1.2152, 0.1825])
Bom Dia, professor.  =>  Vai cair tudo na prova? : tensor(0.6988)
Total sentenças: 4
Total: tensor(1.7721)
Média: tensor(0.4430)


### Comparando palavras

In [ ]:
sentenca1 = "cachorro"
sentenca2 = "gato"
sentenca3 = "avião"
sentenca4 = "ônibus"

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
resultado = pontuacaoSentenca(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)

print("\nSentença 1:", sentenca1)
print("Sentença 3:", sentenca3)
resultado = pontuacaoSentenca(sentenca1, sentenca3)
print("Resultado da relação da sentença 3:", resultado)

print("\nSentença 3:", sentenca3)
print("Sentença 4:", sentenca4)
resultado = pontuacaoSentenca(sentenca3, sentenca4)
print("Resultado da relação da sentença 4:", resultado)

print("\nSentença 3:", sentenca3)
print("Sentença 1:", sentenca1)
resultado = pontuacaoSentenca(sentenca3, sentenca1)
print("Resultado da relação da sentença 1:", resultado)


Sentença 1: cachorro
Sentença 2: gato
Saída: tensor([-0.9021,  0.8146])
Resultado da relação da sentença 2: tensor(-0.0438)

Sentença 1: cachorro
Sentença 3: avião
Saída: tensor([-0.3603,  0.9946])
Resultado da relação da sentença 3: tensor(0.3172)

Sentença 3: avião
Sentença 4: ônibus
Saída: tensor([0.3336, 0.0673])
Resultado da relação da sentença 4: tensor(0.2005)

Sentença 3: avião
Sentença 1: cachorro
Saída: tensor([-0.4827,  0.9095])
Resultado da relação da sentença 1: tensor(0.2134)


### Grupo de palavras relacionadas

In [ ]:
print("Grupo palavras")

# Define uma lista com 4 palavras
texto = ['cachorro',
         'gato',              
         'cavalo',
         'zebra']

total = 0
for i in range(len(texto)-1):    
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Grupo palavras
Saída: tensor([-0.9021,  0.8146])
cachorro  =>  gato : tensor(-0.0438)
Saída: tensor([-0.0915,  1.0238])
gato  =>  cavalo : tensor(0.4662)
Saída: tensor([0.0490, 1.0001])
cavalo  =>  zebra : tensor(0.5246)
Total sentenças: 4
Total: tensor(1.0345)
Média: tensor(0.2586)


### Grupo de palavras relacionadas com ruído

In [ ]:
print("Grupo palavras")

# Define uma lista com 4 palavras
texto = ['cachorro',
         'gato',              
         'avião', #<= Ruído
         'zebra']

total = 0
for i in range(len(texto)-1):    
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Grupo palavras
Saída: tensor([-0.9021,  0.8146])
cachorro  =>  gato : tensor(-0.0438)
Saída: tensor([0.1469, 1.3691])
gato  =>  avião : tensor(0.7580)
Saída: tensor([0.2148, 0.8442])
avião  =>  zebra : tensor(0.5295)
Total sentenças: 4
Total: tensor(1.3313)
Média: tensor(0.3328)


## 7 - Função de previsão da próxima sentença com softmax

In [ ]:
import torch

def previsaoProximaSentenca(entrada, candidato):      
  # Adiciona os tokens especiais ao texto
  texto_marcado = "[CLS] " + entrada + " [SEP] " + candidato + " [SEP]"
  #print("texto_marcado:", texto_marcado)

  # Divide a sentenças em tokens
  texto_tokenizado = tokenizer.tokenize(texto_marcado)    
  #print("texto_tokenizado:", texto_tokenizado

  # Mapeia os tokens em seus índices do vocabulario
  tokens_indexados = tokenizer.convert_tokens_to_ids(texto_tokenizado)
  #print("tokens_indexados:", tokens_indexados)

  # Converte as entradas de lista para tensores do torch
  tokens_tensores = torch.tensor([tokens_indexados])

  # Realiza a predição da sentença
  with torch.no_grad():
	  # Submete o texto ao modela
    outputs = model(tokens_tensores)  
	  # outputs é uma matriz com perdas como o primeiro valor e logits como o segundo
    # outputs[0] = last_hidden_state
    # outputs[0][0] é igual outputs.logits[0]. Logits retorna dois valores, referenciado pelo terceiro índice.      
    #print("Saída:",outputs[0][0])
    
  # Normaliza os pesos das predições nos embeddings e calcula sua probabilidade
  probabilidades  = torch.nn.functional.softmax(outputs[0], dim=1)
  print("Probabilidades:", probabilidades)

  # A função `.item()` retorna apenas o valor Python do tensor.
  # Torch.argmax retorna o índice da posição do maior valor
  previsao = torch.argmax(probabilidades).item()
  #print("Previsão:", previsao)

  return probabilidades , previsao

### Comparando sentenças

In [ ]:
sentenca1 = "O que é uma fila?"
sentenca2 = "Como inserir um elemento em uma fila?" 

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
probabilidade, previsao = previsaoProximaSentenca(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)
print("Probabilidade:", probabilidade)
print("Previsão     :", previsao)

Sentença 1: O que é uma fila?
Sentença 2: Como inserir um elemento em uma fila?
Probabilidades: tensor([[0.5512, 0.4488]])
Resultado da relação da sentença 2: tensor(0.2134)
Probabilidade: tensor([[0.5512, 0.4488]])
Previsão     : 0


In [ ]:
sentenca1 = "O que é uma fila?"
sentenca2 = "Como esvaziar uma pilha?" 
 
print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
probabilidade, previsao = previsaoProximaSentenca(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)
print("Probabilidade:", probabilidade)
print("Previsão     :", previsao)

Sentença 1: O que é uma fila?
Sentença 2: Como esvaziar uma pilha?
Probabilidades: tensor([[0.3691, 0.6309]])
Resultado da relação da sentença 2: tensor(0.2134)
Probabilidade: tensor([[0.3691, 0.6309]])
Previsão     : 1


In [ ]:
sentenca1 = "Hoje está calor."
sentenca2 = "Vamos a praia."
sentenca3 = "O carro estragou."
sentenca4 = "Vou levar a oficina."

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
probabilidade, previsao = previsaoProximaSentenca(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)
print("Probabilidade:", probabilidade)
print("Previsão     :", previsao)

print("\nSentença 1:", sentenca1)
print("Sentença 3:", sentenca3)
probabilidade, previsao = previsaoProximaSentenca(sentenca1, sentenca3)
print("Resultado da relação da sentença 3:", resultado)
print("Probabilidade:", probabilidade)
print("Previsão     :", previsao)

print("\nSentença 3:", sentenca3)
print("Sentença 4:", sentenca4)
probabilidade, previsao = previsaoProximaSentenca(sentenca3, sentenca4)
print("Resultado da relação da sentença 4:", resultado)
print("Probabilidade:", probabilidade)
print("Previsão     :", previsao)

print("\nSentença 3:", sentenca3)
print("Sentença 1:", sentenca1)
probabilidade, previsao = previsaoProximaSentenca(sentenca3, sentenca1)
print("Resultado da relação da sentença 1:", resultado)
print("Probabilidade:", probabilidade)
print("Previsão     :", previsao)


Sentença 1: Hoje está calor.
Sentença 2: Vamos a praia.
Probabilidades: tensor([[0.8249, 0.1751]])
Resultado da relação da sentença 2: tensor(0.2134)
Probabilidade: tensor([[0.8249, 0.1751]])
Previsão     : 0

Sentença 1: Hoje está calor.
Sentença 3: O carro estragou.
Probabilidades: tensor([[0.3879, 0.6121]])
Resultado da relação da sentença 3: tensor(0.2134)
Probabilidade: tensor([[0.3879, 0.6121]])
Previsão     : 1

Sentença 3: O carro estragou.
Sentença 4: Vou levar a oficina.
Probabilidades: tensor([[0.8788, 0.1212]])
Resultado da relação da sentença 4: tensor(0.2134)
Probabilidade: tensor([[0.8788, 0.1212]])
Previsão     : 0

Sentença 3: O carro estragou.
Sentença 1: Hoje está calor.
Probabilidades: tensor([[0.5741, 0.4259]])
Resultado da relação da sentença 1: tensor(0.2134)
Probabilidade: tensor([[0.5741, 0.4259]])
Previsão     : 0


### Comparando uma sequência de sentenças 1

In [ ]:
# Define um texto
texto = ['Fui de carro.',
         'O caminhão tombou.',              
         'O ônibus atrasou.']

for i in range(len(texto)-1):
    probabilidade, previsao = previsaoProximaSentenca(texto[i], texto[i+1])
    print(texto[i], " => ",texto[i+1], ":", probabilidade)
    print("    Probabilidade:", probabilidade)
    print("    Previsao     :", previsao)

Probabilidades: tensor([[0.7283, 0.2717]])
Fui de carro.  =>  O caminhão tombou. : tensor([[0.7283, 0.2717]])
    Probabilidade: tensor([[0.7283, 0.2717]])
    Previsao     : 0
Probabilidades: tensor([[0.6559, 0.3441]])
O caminhão tombou.  =>  O ônibus atrasou. : tensor([[0.6559, 0.3441]])
    Probabilidade: tensor([[0.6559, 0.3441]])
    Previsao     : 0


### Comparando uma sequência de sentenças 2

In [ ]:
# Define um texto com a permutação das sentenças do texto original
texto = ['O gato saiu correndo.',   
             'O cachorro latiu.',             
             'O passáro está no ninho.']    

for i in range(len(texto)-1):
    probabilidade, previsao = previsaoProximaSentenca(texto[i], texto[i+1])
    print(texto[i], " => ",texto[i+1], ":", probabilidade)
    print("    Probabilidade:", probabilidade)
    print("    Previsao     :", previsao)

Probabilidades: tensor([[0.3580, 0.6420]])
O gato saiu correndo.  =>  O cachorro latiu. : tensor([[0.3580, 0.6420]])
    Probabilidade: tensor([[0.3580, 0.6420]])
    Previsao     : 1
Probabilidades: tensor([[0.5268, 0.4732]])
O cachorro latiu.  =>  O passáro está no ninho. : tensor([[0.5268, 0.4732]])
    Probabilidade: tensor([[0.5268, 0.4732]])
    Previsao     : 0


### Comparando texto original com permutado

In [ ]:
print("Texto Original")

# Define um texto com 4 sentenças
texto_original = ['Bom Dia, professor.',
             'Qual o conteúdo da prova?',              
             'Vai cair tudo na prova?',
             'Aguardo uma resposta, João.']

texto = texto_original

for i in range(len(texto)-1):
    probabilidade, previsao = previsaoProximaSentenca(texto[i], texto[i+1])
    print(texto[i], " => ",texto[i+1], ":", probabilidade)
    print("    Probabilidade:", probabilidade)
    print("    Previsao     :", previsao)

print("\nTexto permutado")
# Define um texto com a permutação das sentenças do texto original
texto = [texto_original[3],   # 'Aguardo uma resposta, João.',
             texto_original[1],             # 'Qual o conteúdo da prova?',              
             texto_original[0],             # 'Vai cair tudo na prova?',
             texto_original[2]]             # 'Bom Dia, professor.']                 


for i in range(len(texto)-1):
    probabilidade, previsao = previsaoProximaSentenca(texto[i], texto[i+1])
    print(texto[i], " => ",texto[i+1], ":", probabilidade)
    print("    Probabilidade:", probabilidade)
    print("    Previsao     :", previsao)

Texto Original
Probabilidades: tensor([[0.8566, 0.1434]])
Bom Dia, professor.  =>  Qual o conteúdo da prova? : tensor([[0.8566, 0.1434]])
    Probabilidade: tensor([[0.8566, 0.1434]])
    Previsao     : 0
Probabilidades: tensor([[0.5657, 0.4343]])
Qual o conteúdo da prova?  =>  Vai cair tudo na prova? : tensor([[0.5657, 0.4343]])
    Probabilidade: tensor([[0.5657, 0.4343]])
    Previsao     : 0
Probabilidades: tensor([[0.9299, 0.0701]])
Vai cair tudo na prova?  =>  Aguardo uma resposta, João. : tensor([[0.9299, 0.0701]])
    Probabilidade: tensor([[0.9299, 0.0701]])
    Previsao     : 0

Texto permutado
Probabilidades: tensor([[0.8850, 0.1150]])
Aguardo uma resposta, João.  =>  Qual o conteúdo da prova? : tensor([[0.8850, 0.1150]])
    Probabilidade: tensor([[0.8850, 0.1150]])
    Previsao     : 0
Probabilidades: tensor([[0.7519, 0.2481]])
Qual o conteúdo da prova?  =>  Bom Dia, professor. : tensor([[0.7519, 0.2481]])
    Probabilidade: tensor([[0.7519, 0.2481]])
    Previsao     : 0


### Comparando palavras

In [ ]:
sentenca1 = "cachorro"
sentenca2 = "gato"
sentenca3 = "avião"
sentenca4 = "ônibus"

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
probabilidade, previsao = previsaoProximaSentenca(sentenca1, sentenca2)
print("Probabilidade:", probabilidade)
print("Previsão     :", previsao)

print("\nSentença 1:", sentenca1)
print("Sentença 3:", sentenca3)
probabilidade, previsao = previsaoProximaSentenca(sentenca1, sentenca3)
print("Resultado da relação da sentença 3:", probabilidade)
print("Previsão     :", previsao)

print("\nSentença 3:", sentenca3)
print("Sentença 4:", sentenca4)
probabilidade, previsao = previsaoProximaSentenca(sentenca3, sentenca4)
print("Resultado da relação da sentença 4:", probabilidade)
print("Previsão     :", previsao)

print("\nSentença 3:", sentenca3)
print("Sentença 1:", sentenca1)
probabilidade, previsao = previsaoProximaSentenca(sentenca3, sentenca1)
print("Resultado da relação da sentença 1:", probabilidade)
print("Previsão     :", previsao)

Sentença 1: cachorro
Sentença 2: gato
Probabilidades: tensor([[0.1523, 0.8477]])
Probabilidade: tensor([[0.1523, 0.8477]])
Previsão     : 1

Sentença 1: cachorro
Sentença 3: avião
Probabilidades: tensor([[0.2051, 0.7949]])
Resultado da relação da sentença 3: tensor([[0.2051, 0.7949]])
Previsão     : 1

Sentença 3: avião
Sentença 4: ônibus
Probabilidades: tensor([[0.5662, 0.4338]])
Resultado da relação da sentença 4: tensor([[0.5662, 0.4338]])
Previsão     : 0

Sentença 3: avião
Sentença 1: cachorro
Probabilidades: tensor([[0.1991, 0.8009]])
Resultado da relação da sentença 1: tensor([[0.1991, 0.8009]])
Previsão     : 1


### Grupo de palavras relacionadas

In [ ]:
print("Grupo palavras")

# Define uma lista com 4 palavras
texto = ['cachorro',
         'gato',              
         'cavalo',
         'zebra']

for i in range(len(texto)-1):    
    probabilidade, previsao = previsaoProximaSentenca(texto[i], texto[i+1])
    print(texto[i], " => ",texto[i+1], ":", probabilidade)
    print("    Probabilidade:", probabilidade)
    print("    Previsao     :", previsao)

Grupo palavras
Probabilidades: tensor([[0.1523, 0.8477]])
cachorro  =>  gato : tensor([[0.1523, 0.8477]])
    Probabilidade: tensor([[0.1523, 0.8477]])
    Previsao     : 1
Probabilidades: tensor([[0.2469, 0.7531]])
gato  =>  cavalo : tensor([[0.2469, 0.7531]])
    Probabilidade: tensor([[0.2469, 0.7531]])
    Previsao     : 1
Probabilidades: tensor([[0.2787, 0.7213]])
cavalo  =>  zebra : tensor([[0.2787, 0.7213]])
    Probabilidade: tensor([[0.2787, 0.7213]])
    Previsao     : 1


### Grupo de palavras relacionadas com ruído

In [ ]:
print("Grupo palavras")

# Define uma lista com 4 palavras
texto = ['cachorro',
         'gato',              
         'avião', #<= Ruído
         'zebra']

for i in range(len(texto)-1):    
    probabilidade, previsao = previsaoProximaSentenca(texto[i], texto[i+1])
    print(texto[i], " => ",texto[i+1], ":", probabilidade)
    print("    Probabilidade:", probabilidade)
    print("    Previsao     :", previsao)

Grupo palavras
Probabilidades: tensor([[0.1523, 0.8477]])
cachorro  =>  gato : tensor([[0.1523, 0.8477]])
    Probabilidade: tensor([[0.1523, 0.8477]])
    Previsao     : 1
Probabilidades: tensor([[0.2276, 0.7724]])
gato  =>  avião : tensor([[0.2276, 0.7724]])
    Probabilidade: tensor([[0.2276, 0.7724]])
    Previsao     : 1
Probabilidades: tensor([[0.3476, 0.6524]])
avião  =>  zebra : tensor([[0.3476, 0.6524]])
    Probabilidade: tensor([[0.3476, 0.6524]])
    Previsao     : 1
